In [1]:
import os
import numpy as np
import gym
from gym import wrappers
import pybullet_envs

In [10]:
class Hp():
    def __init__(self):
        self.nb_steps=2
        self.episode_length=1000
        self.learning_rate=0.02
        self.nb_directions=16
        self.nb_best_directions=4
        assert self.nb_best_directions<=self.nb_directions
        self.noise=0.03
        self.seed=1
        self.env_name='HalfCheetahBulletEnv-v0'

In [3]:
class Normalizer():
    
    def __init__(self,nb_inputs):
        self.n=np.zeros(nb_inputs)
        self.mean=np.zeros(nb_inputs)
        self.mean_diff = np.zeros(nb_inputs)
        self.var=np.zeros(nb_inputs)
        
    def observe(self,x):
        self.n+=1.
        last_mean=self.mean.copy()
        self.mean+=(x-self.mean)/self.n
        self.mean_diff+=(x-last_mean)*(x-self.mean)
        self.var=(self.mean_diff/self.n).clip(min=1e-2)
            
    def normalize(self,inputs):
        obs_mean=self.mean
        obs_std=np.sqrt(self.var)
        return (inputs-obs_mean)/obs_std

In [4]:
class Policy():
    def __init__(self,input_size,output_size):
        self.theta=np.zeros((output_size,input_size))
    
    def evaluate(self,hp,input,delta=None,direction=None):
        if direction is None:
            return self.theta.dot(input)
        elif direction=="positive":
            return(self.theta+hp.noise*delta).dot(input)
        else:
            return(self.theta-hp.noise*delta).dot(input)
    
    def sample_deltas(self,hp):
        return[np.random.randn(*self.theta.shape) for _ in range(hp.nb_directions)]
    
    def update(self,hp,rollouts,sigma_r):
        step=np.zeros(self.theta.shape)
        for r_pos,r_neg,d in rollouts:
            step+=(r_pos-r_neg)*d
            self.theta+=hp.learning_rate/(hp.nb_best_directions*sigma_r)*step
        return self.theta
            

In [5]:
def explore(hp,env,normalizer,policy,direction=None,delta=None):
    state=env.reset()
    done=False
    num_plays=0.
    sum_rewards=0.
    while not done and num_plays<hp.episode_length:
        #env.render()
        normalizer.observe(state)
        state=normalizer.normalize(state)
        action=policy.evaluate(hp,state,delta,direction)
        state,reward,done,_=env.step(action)
        reward=max(min(reward,1),-1)
        sum_rewards+=reward
        num_plays+=1
    return sum_rewards

In [6]:
def train(env,policy,normalizer,hp):
    for step in range(1):
        deltas=policy.sample_deltas(hp)
        positive_rewards=[0]*hp.nb_directions
        negative_rewards=[0]*hp.nb_directions

        for k in range(hp.nb_directions):
            positive_rewards[k]=explore(hp,env,normalizer,policy,direction="positive",delta=deltas[k])

        for k in range(hp.nb_directions):
            negative_rewards[k]=explore(hp,env,normalizer,policy,direction="negative",delta=deltas[k])

        all_rewards=np.array(positive_rewards+negative_rewards)
        sigma_r=all_rewards.std()

        scores={k:max(r_pos,r_neg) for k,(r_pos,r_neg) in enumerate(zip(positive_rewards,negative_rewards))}
        order=sorted(scores.keys(),key=lambda x:scores[x], reverse=True)[:hp.nb_best_directions]
        rollouts=[(positive_rewards[k],negative_rewards[k],deltas[k]) for k in order]

        if step==0:
            weights=np.array(policy.update(hp,rollouts,sigma_r))
            reward_evaluation=np.array(explore(hp,env,normalizer,policy))
        else:
            weights=np.vstack((weights,policy.update(hp,rollouts,sigma_r)))
            reward_evaluation=np.vstack((reward_evaluation,explore(hp,env,normalizer,policy)))

        
        print('Reward:',reward_evaluation,'weights:',weights,sigma_r)
    return reward_evaluation,weights,sigma_r

In [7]:
class co_op():
    def __init__(self,hp_seed,env,weights=None,update=False):
        #self.test=hp_seed
        self.hp=Hp()
        self.hp.seed=hp_seed
        #self.hp.seed=int(time.time())
        np.random.seed(self.hp.seed)
        #self.env=gym.make(self.hp.env_name)
        #self.work_dir=mkdir('exp','brs{}'.format(hp_seed))
        #self.monitor_dir=mkdir(self.work_dir,'monitor')
        #self.env=wrappers.Monitor(self.env,self.monitor_dir,force=True)
        self.nb_inputs=env.observation_space.shape[0]
        self.nb_outputs=env.action_space.shape[0]
        self.policy=Policy(self.nb_inputs,self.nb_outputs)
        if update:
            self.policy.theta=weights
        self.normalizer=Normalizer(self.nb_inputs)
    #return env,policy,normalizer
    #print(hp.nb_directions)
        self.reward_evaluation,self.weights,self.sigma_r=train(env,self.policy,self.normalizer,self.hp)
    #return reward_evaluation,weights

In [8]:
import os
def mkdir(base,name):
    path=os.path.join(base,name)
    if not os.path.exists(path):
        os.makedirs(path)
    return path
#work_dir=mkdir('exp','brs')
#monitor_dir=mkdir(work_dir,'monitor')
#work_dir1=mkdir('exp','brs1')
#monitor_dir1=mkdir(work_dir1,'monitor')

In [11]:
import time

hp=Hp()
np.random.seed(hp.seed)
env1=gym.make(hp.env_name)
work_dir1=mkdir('exp','brs1')
monitor_dir1=mkdir(work_dir1,'monitor')
hp.seed=2
np.random.seed(hp.seed)
env2=gym.make(hp.env_name)
work_dir2=mkdir('exp','brs2')
monitor_dir2=mkdir(work_dir2,'monitor')
Weights_1=np.zeros([env1.action_space.shape[0],env1.observation_space.shape[0]])
Weights_2=np.zeros([env2.action_space.shape[0],env2.observation_space.shape[0]])


for it in range(10):
    Weighting_1=np.zeros([3])
    Weighting_2=np.zeros([3])
    Weighting_1d=np.zeros([3])
    Weighting_2d=np.zeros([3])
    local_1=np.zeros([env1.action_space.shape[0],env1.observation_space.shape[0]])
    local_2=np.zeros([env2.action_space.shape[0],env2.observation_space.shape[0]])
    local_1d=np.zeros([env1.action_space.shape[0],env1.observation_space.shape[0]])
    local_2d=np.zeros([env2.action_space.shape[0],env2.observation_space.shape[0]])
    #env1=wrappers.Monitor(env1,monitor_dir1,force=False)
    #env2=wrappers.Monitor(env2,monitor_dir2,force=False)
    for j in range(10):
        print(it,j)
        if j==0 and it==0:
            co_op_1=co_op(1,env1)
            co_op_2=co_op(2,env2)
            creward_1=0
            creward_2=0
        elif j!=0:
            co_op_1=co_op(1,env1,weights_1c,True)
            co_op_2=co_op(2,env2,weights_2c,True)
        else:
            co_op_1=co_op(1,env1,weights_1,True)
            co_op_2=co_op(2,env2,weights_2,True)
        weights_1c=(0.1*co_op_2.weights)+(0.9*co_op_1.weights)
        weights_2c=(0.1*co_op_1.weights)+(0.9*co_op_2.weights)
        if co_op_1.sigma_r<0.01:
            co_op_1.sigma_r=0.01
        if co_op_2.sigma_r<0.01:
            co_op_2.sigma_r=0.01
        creward_1+=(np.exp(co_op_1.reward_evaluation/250)*np.power(1.2,j))/(co_op_1.sigma_r)
        creward_2+=(np.exp(co_op_2.reward_evaluation/250)*np.power(1.2,j))/(co_op_2.sigma_r)
        Weighting_1=np.vstack((Weighting_1,[np.exp((co_op_1.reward_evaluation)/250),co_op_1.sigma_r,np.power(1.2,j)]))
        Weighting_2=np.vstack((Weighting_2,[np.exp((co_op_2.reward_evaluation)/250),co_op_2.sigma_r,np.power(1.2,j)]))
        local_1=np.vstack((local_1,weights_1c))
        local_2=np.vstack((local_2,weights_2c))
    #env1=wrappers.Monitor(env1,monitor_dir1,force=False)
    #env2=wrappers.Monitor(env2,monitor_dir2,force=False)
    for j in range(10):
        print(it,j)
        if j==0 and it==0:
            co_op_1d=co_op(1,env1)
            co_op_2d=co_op(2,env2)
            creward_1d=0
            creward_2d=0
        elif j!=0:
            co_op_1d=co_op(1,env1,weights_1d,True)
            co_op_2d=co_op(2,env2,weights_2d,True)
        else:
            co_op_1d=co_op(1,env1,weights_1,True)
            co_op_2d=co_op(2,env2,weights_2,True)
        weights_1d=(1.0*co_op_1d.weights)
        weights_2d=(1.0*co_op_2d.weights)
        if co_op_1d.sigma_r<0.01:
            co_op_1d.sigma_r=0.01
        if co_op_2d.sigma_r<0.01:
            co_op_2d.sigma_r=0.01
        creward_1d+=(np.exp(co_op_1d.reward_evaluation/250)*np.power(1.2,j))/(co_op_1d.sigma_r)
        creward_2d+=(np.exp(co_op_2d.reward_evaluation/250)*np.power(1.2,j))/(co_op_2d.sigma_r)
        Weighting_1d=np.vstack((Weighting_1d,[np.exp((co_op_1d.reward_evaluation)/250),co_op_1d.sigma_r,np.power(1.2,j)]))
        Weighting_2d=np.vstack((Weighting_2d,[np.exp((co_op_2d.reward_evaluation)/250),co_op_2d.sigma_r,np.power(1.2,j)]))
        local_1d=np.vstack((local_1d,weights_1d))
        local_2d=np.vstack((local_2d,weights_2d))
    std_1=Weighting_1[1:,0].std()
    std_2=Weighting_2[1:,0].std()
    std_1d=Weighting_1d[1:,0].std()
    std_2d=Weighting_2d[1:,0].std()
    if std_1<0.01:
        std_1=0.01
    if std_2<0.01:
        std_2=0.01
    if std_1d<0.01:
        std_1d=0.01
    if std_2d<0.01:
        std_2d=0.01
    creward_1=creward_1/std_1
    creward_2=creward_2/std_2
    creward_1d=creward_1d/std_1d
    creward_2d=creward_2d/std_2d
    if creward_1+creward_2>creward_1d+creward_2d:
        avg_weighting_1=(Weighting_1[1:,0]*Weighting_1[1:,2])/Weighting_1[1:,1]
        avg_weighting_2=(Weighting_2[1:,0]*Weighting_2[1:,2])/Weighting_2[1:,1]
        local_1=local_1[env1.action_space.shape[0]:,:]
        local_2=local_2[env2.action_space.shape[0]:,:]
        local_1l=local_1.reshape([-1,10])
        local_2l=local_2.reshape([-1,10])
        #local_1=local_1.reshape([env1.action_space.shape[0],env1.observation_space.shape[0]])
        #local_2=local_2.reshape([env2.action_space.shape[0],env2.observation_space.shape[0]])
        weights_1=np.average(local_1l,axis=1,weights=avg_weighting_1)
        weights_2=np.average(local_2l,axis=1,weights=avg_weighting_2)
        weights_1=weights_1.reshape([env1.action_space.shape[0],env1.observation_space.shape[0]])
        weights_2=weights_2.reshape([env1.action_space.shape[0],env1.observation_space.shape[0]])
        
        c=1

    else:
        avg_weighting_1d=(Weighting_1d[1:,0]*Weighting_1d[1:,2])/Weighting_1d[1:,1]
        avg_weighting_2d=(Weighting_2d[1:,0]*Weighting_2d[1:,2])/Weighting_2d[1:,1]
        local_1d=local_1d[env1.action_space.shape[0]:,:]
        local_2d=local_2d[env2.action_space.shape[0]:,:]
        local_1dl=local_1d.reshape([-1,10])
        local_2dl=local_2d.reshape([-1,10])
        #local_1d=local_1d.reshape([env1.action_space.shape[0],env1.observation_space.shape[0]])
        #local_2d=local_2d.reshape([env2.action_space.shape[0],env2.observation_space.shape[0]])
        weights_1=np.average(local_1dl,axis=1,weights=avg_weighting_1d)
        weights_2=np.average(local_2dl,axis=1,weights=avg_weighting_2d)
        weights_1=weights_1.reshape([env1.action_space.shape[0],env1.observation_space.shape[0]])
        weights_2=weights_2.reshape([env1.action_space.shape[0],env1.observation_space.shape[0]])
        c=0
    Weights_1=np.vstack((Weights_1,weights_1))
    Weights_2=np.vstack((Weights_2,weights_2))
    print('Co_operation: ',c,'creward_1',creward_1,'creward_2',creward_2,'creward_1d',creward_1d,'creward_2d',creward_2d)
    creward_1=0
    creward_2=0
    creward_1d=0
    creward_2d=0





WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WalkerBase::__init__ start
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WalkerBase::__init__ start
0 0
Reward: -569.4133560693544 weights: [[-0.08636579 -0.08009395 -0.06327591  0.04626613 -0.0847776   0.01136787
  -0.09953647 -0.1454941  -0.00669371  0.04380159  0.07235349 -0.00338654
  -0.10144298  0.08635581  0.077788   -0.00495757 -0.0115437  -0.09929605
   0.09012688 -0.005657   -0.08981253 -0.01437197 -0.03125253 -0.10081363
   0.07105217 -0.04769717]
 [ 0.03028551  0.03960193  0.11063752 -0.04623872  0.10270012  0.02057326
  -0.028576    0.06932047  0.00169121  0.01765271  0.05076964  0.1768241
  -0.05039313 -0.01551829  0.11744176  0

Reward: -989.4575919360791 weights: [[-0.0644311  -0.25342442 -0.09266261  0.07868055 -0.04557029  0.2046381
  -0.21046255 -0.33970255 -0.07710507  0.11137695  0.09203856 -0.07757127
  -0.08558563  0.00668403  0.08051693  0.10907112  0.15202239 -0.06336154
   0.28917551  0.13065578  0.02980744 -0.10835416 -0.18280068 -0.08068818
  -0.03817839 -0.01813334]
 [-0.25399397 -0.13233312  0.31182197 -0.22469867  0.04771828  0.20842542
  -0.17607418  0.08329979 -0.04118263 -0.14977028 -0.05660753  0.32779123
  -0.15367714 -0.1115786   0.13902041  0.19911184  0.1665403   0.02059794
  -0.09970283 -0.04794829  0.0812675  -0.11341813  0.00769297  0.14920364
   0.02648685 -0.11669354]
 [ 0.29994034  0.12119151 -0.01192067  0.06045414  0.13427961 -0.02455154
   0.00317526  0.04399099  0.12257392  0.19667812  0.00873775  0.07292759
   0.00914791  0.17930531 -0.08983873  0.06671143  0.28797793  0.09373123
  -0.07331036 -0.02628803 -0.02085652 -0.36069725 -0.08656218  0.35130373
   0.07772698 -0.349346

Reward: 156.4209053272795 weights: [[-0.09382463 -0.29872923 -0.08837837  0.14420321 -0.04673054  0.10413282
  -0.37279512 -0.4372674  -0.0311125   0.30131811  0.05475312  0.00323071
  -0.2192929   0.17695668  0.03869009  0.1028068   0.21192797 -0.02839705
   0.25013143  0.15266813 -0.11295635 -0.09064134 -0.3588661  -0.28370647
  -0.09295295 -0.07027869]
 [-0.17619733 -0.02006319  0.53390418 -0.11123821  0.11819525  0.28934193
  -0.06626944  0.17169834 -0.00966138 -0.24222211  0.03339832  0.32152242
  -0.31993008 -0.25445442  0.34901434  0.29469311  0.11587428  0.05723384
  -0.25281075  0.06170378 -0.09504551 -0.11315748  0.07728959  0.25808278
  -0.08876119 -0.18598867]
 [ 0.37301271  0.24101215 -0.04659774 -0.06321891  0.09949175 -0.1828157
  -0.18913055  0.23989277  0.39801407  0.09751596 -0.03333396  0.2132515
   0.10029644  0.28515391 -0.22157353  0.10086357  0.28273299  0.2031161
   0.02144178 -0.07724531  0.08009885 -0.44395613 -0.13518483  0.43635432
   0.16772544 -0.28308085]

Reward: 11.941950581015652 weights: [[-0.11965387 -0.50177942 -0.25040513  0.12481575 -0.03965905 -0.04982602
  -0.41313494 -0.63076512  0.04086469  0.46551017  0.01020779 -0.01368129
  -0.29292874  0.27505881  0.14103871  0.07491022  0.1960452  -0.25573576
   0.32695076  0.07773385 -0.1852054  -0.00480957 -0.48971181 -0.28199237
  -0.01592979 -0.11115009]
 [-0.25151872  0.0348947   0.68723829  0.05887954  0.17632226  0.30627728
  -0.06904743  0.13249213  0.04153038 -0.2691509   0.0169581   0.58662707
  -0.35719669 -0.27512405  0.49898653  0.3799986   0.04228121  0.11262096
  -0.4082028   0.04065249 -0.02907283  0.01746484  0.1475194   0.23739901
  -0.05893507 -0.25602439]
 [ 0.35864862  0.33977389 -0.16990328  0.0208729   0.12414483 -0.26999256
  -0.25008689  0.3693246   0.39519277  0.2868377  -0.06528195  0.21510652
   0.16318593  0.47617532 -0.17216412  0.21441377  0.4956182   0.17780178
   0.13617167 -0.19018215  0.19543958 -0.516207    0.03200419  0.56873969
   0.38747082 -0.27809

Reward: -932.9714937607341 weights: [[-1.46216318e-02 -4.09664917e-01 -1.24264408e-01  2.82788780e-01
  -6.50485333e-02 -1.33131505e-01 -3.03413734e-01 -4.49220277e-01
  -8.49547572e-02  4.95989334e-01 -1.01113826e-02  7.33833301e-02
  -3.14625884e-01  1.43598220e-01 -4.08835230e-02 -2.24237358e-02
   1.60943460e-01 -4.64517920e-02  2.37549826e-01  1.22413769e-01
  -2.67992539e-01  1.20321784e-02 -2.94844897e-01 -3.55807568e-01
   1.16652554e-01 -9.46042857e-02]
 [-2.35962406e-01  8.91623229e-02  4.25971176e-01  2.58481074e-02
   2.09547290e-01  1.76139132e-01  1.17387202e-01  2.65249766e-02
   7.61322206e-02 -1.42368775e-01  5.67062248e-02  4.91302975e-01
  -3.81200980e-01 -2.08176472e-01  2.55636234e-01  2.32921343e-01
   6.72536152e-02  1.86873677e-01 -4.07825689e-01 -3.24336876e-02
  -1.85681781e-01 -4.49681079e-02  1.64893046e-01  1.44673928e-01
   1.00790126e-01 -6.00380518e-04]
 [ 3.02639329e-01  3.08513671e-01 -2.80972449e-01  2.35934974e-01
   1.87845333e-01 -1.43928235e-01 -1

...

Reward: -938.1947517523232 weights: [[-2.07369750e-02  2.24022243e-02  4.41241539e-02  5.07913253e-02
   9.00910387e-02 -2.45600793e-01  8.07458571e-02  2.03897021e-01
  -3.41859198e-01  6.58374949e-03  4.37344552e-02  1.49479532e-01
  -1.74460406e-01 -1.05931089e-01 -5.00108926e-01  7.20608320e-03
  -9.64736957e-02  1.78978585e-01 -9.83044220e-02  2.04337899e-01
   1.60980674e-01 -5.23263021e-02 -9.07264936e-02 -8.85366517e-02
  -1.81684327e-02  1.95197746e-02]
 [-1.51305776e-04  1.68528596e-01 -8.65534851e-02  2.70432655e-01
   6.75193286e-02 -6.83906716e-02  2.83406591e-01 -6.76028247e-02
   2.15340590e-02  1.07828655e-01 -1.12918599e-01 -3.04978468e-02
  -1.08365079e-01 -3.71758179e-02  1.81669363e-01 -2.05219453e-01
  -1.44205619e-02 -7.58663958e-03 -4.76517411e-03 -1.75434648e-01
   1.03210003e-02 -1.97627997e-02  4.76163421e-02 -2.70758232e-01
   1.25912503e-02  2.43426770e-01]
 [-1.53577464e-01  1.72411270e-01 -1.60275445e-01  4.15327857e-01
   1.67546474e-01 -1.18607715e-01  2

Reward: -797.0329704579509 weights: [[ 0.13242773  0.16378682  0.14511319 -0.07047716  0.10419829 -0.24300089
   0.35180464  0.35425542 -0.33479813 -0.23688989  0.16794806  0.03979767
  -0.06574233 -0.1994696  -0.4972666   0.03409037 -0.12474246  0.23578426
  -0.22266252  0.18395725  0.1684482  -0.00926621  0.19899985  0.10139864
   0.07258934  0.04053834]
 [-0.00206496 -0.04830297 -0.34474048  0.1611664  -0.0126699  -0.17788332
   0.20366243 -0.15820923 -0.11007041  0.22170529 -0.21536028 -0.20232461
   0.15003995  0.12003095 -0.1589721  -0.32952812 -0.01650615  0.06287108
   0.1450854  -0.2987268   0.04280657  0.08150667  0.00536578 -0.26545029
   0.04260949  0.39804699]
 [-0.29245311  0.00420395 -0.13089252  0.42824566  0.20798849  0.04140443
   0.19263195  0.15148847 -0.3260857  -0.09976077  0.25718574 -0.13062215
  -0.07744129  0.10730853  0.33403019  0.01456602 -0.23980231 -0.05294023
  -0.03689356  0.04621975 -0.09162171  0.14109805  0.02896057 -0.00981231
  -0.44291349  0.25729

Reward: -938.9694216491229 weights: [[ 0.27892635  0.06204497  0.11504216 -0.16364897  0.09699204 -0.29075304
   0.51887291  0.31316143 -0.46942959 -0.23117522  0.39670675  0.0151428
  -0.0565972  -0.28291913 -0.27408875  0.04217386 -0.09526515  0.35778684
  -0.24962977  0.27987853  0.20489824  0.26491049  0.12215507 -0.04833276
   0.09590541  0.12783328]
 [-0.19147793 -0.02185876 -0.44076557  0.12168596  0.04810744 -0.21085759
   0.18422959 -0.30491221 -0.00716139  0.44397096 -0.02413773 -0.15993829
   0.01680315  0.18514709 -0.16493144 -0.41401108  0.029517    0.07446781
   0.30255162 -0.46880423 -0.01440614  0.1135219   0.02817841 -0.12684186
   0.00616733  0.22020848]
 [-0.22614164 -0.04464272  0.01679984  0.61390424  0.48720168  0.19400215
   0.18941766  0.21633471 -0.20171997  0.10611866  0.42840939 -0.13353192
  -0.13067952  0.03701767  0.27603659  0.10836081 -0.35259229 -0.13194598
  -0.11477975  0.03590939 -0.08151944 -0.02942296 -0.0626928   0.02704526
  -0.56209759  0.265033

Reward: 349.83883947443223 weights: [[ 0.08746556 -0.10504216  0.06695427  0.00843033 -0.19559454  0.01176069
  -0.01439542 -0.08940263 -0.09029569  0.3468475   0.01389335  0.32135531
  -0.28504662  0.48700518  0.02642977  0.06691562  0.18854792  0.30122143
  -0.13910733  0.14039231 -0.12831821  0.07024333 -0.20071746 -0.20115946
   0.1720498  -0.2063451 ]
 [-0.08331546 -0.19810191  0.05985335 -0.2083863   0.13951084  0.06293012
   0.19551241 -0.03945154 -0.14507254  0.5261526   0.23641095  0.05875464
  -0.10819668  0.21153358 -0.44756888  0.11916422 -0.08413002  0.10003306
   0.05687715  0.06743825 -0.23966737  0.17720267  0.0078219   0.3589885
   0.07266336  0.13662805]
 [ 0.04559174  0.15289469 -0.19001424 -0.00837664  0.03044656  0.15600391
   0.30345588  0.00242058  0.07549615  0.11532269 -0.08119693  0.1558849
  -0.13872678 -0.17737991  0.41361797  0.3057062  -0.02801396 -0.10679144
  -0.16670347  0.04892389  0.4208323  -0.16699377  0.03753904  0.10448993
   0.02963226  0.3541450

Reward: -745.6465378071844 weights: [[-0.048135   -0.16297606  0.08255599 -0.06688099 -0.25636309 -0.07292879
   0.08303285 -0.18863022 -0.18767459  0.30936139  0.08283387  0.37000237
  -0.31332678  0.56697494 -0.0598408   0.07475012  0.34835308  0.42165323
  -0.12168014  0.04743682 -0.12517902 -0.02802957 -0.40131542 -0.15527002
   0.21550576 -0.22995452]
 [-0.04324993 -0.30352588  0.16287324 -0.29650384  0.15593531  0.08907463
   0.28201849  0.05926491 -0.1359548   0.51736693  0.13234929  0.06075603
  -0.15416647  0.15846028 -0.46853458  0.1258965  -0.12295417  0.06895862
   0.04405565 -0.02003854 -0.38450727  0.26155951  0.03430537  0.49309882
  -0.03187932  0.22024706]
 [ 0.01105623  0.32147921 -0.22353828 -0.02706764  0.05580008  0.18339603
   0.30303002  0.04750025  0.12037568  0.16055424 -0.03288861  0.12499627
  -0.19315467 -0.1746909   0.35307736  0.41981191 -0.04525257 -0.14426309
  -0.0987098   0.05536735  0.51428077 -0.19781422  0.16299202  0.06862663
  -0.14409739  0.29496

In [17]:
0.0001.clip(min=1e-2)

AttributeError: 'float' object has no attribute 'clip'

In [18]:
a=np.ones([3,3])

In [19]:
a=a*0.00001

In [20]:
a

array([[1.e-05, 1.e-05, 1.e-05],
       [1.e-05, 1.e-05, 1.e-05],
       [1.e-05, 1.e-05, 1.e-05]])

In [21]:
a.clip(min=1e-2)

array([[0.01, 0.01, 0.01],
       [0.01, 0.01, 0.01],
       [0.01, 0.01, 0.01]])

In [22]:
a=0.00005
a.clip(1e-2)

AttributeError: 'float' object has no attribute 'clip'

In [40]:
np.exp(2)

7.38905609893065

In [41]:
np.random.seed(1)

In [42]:
np.random.rand(3,2)

array([[4.17022005e-01, 7.20324493e-01],
       [1.14374817e-04, 3.02332573e-01],
       [1.46755891e-01, 9.23385948e-02]])

In [43]:
np.random.seed(1)

In [44]:
np.random.rand(3,2)

array([[4.17022005e-01, 7.20324493e-01],
       [1.14374817e-04, 3.02332573e-01],
       [1.46755891e-01, 9.23385948e-02]])

In [45]:
np.random.rand(3,2)

array([[0.18626021, 0.34556073],
       [0.39676747, 0.53881673],
       [0.41919451, 0.6852195 ]])

In [54]:
a=np.random.rand(3,2)

In [55]:
a

array([[0.14038694, 0.19810149],
       [0.80074457, 0.96826158],
       [0.31342418, 0.69232262]])

In [65]:
a[:,0]

array([0.14038694, 0.80074457, 0.31342418])

In [61]:
a[:,0]*a[:,1]

array([0.02781086, 0.7753302 , 0.21699065])

In [60]:
a[0]

array([0.14038694, 0.19810149])

In [62]:
0.14*0.19

0.026600000000000002

In [63]:
0.8*0.96

0.768

In [64]:
a[0,0]*a[0,1]

0.02781086158378325

In [63]:
data = range(1,5)

In [64]:
data = np.arange(6).reshape((3,2))

In [68]:
np.average(data, axis=1, weights=[1./4, 3./4])

array([0.75, 2.75, 4.75])

In [56]:
datat=np.zeros([26,5])

In [59]:
datat=np.vstack((datat,datat))

In [60]:
datat.shape

(52, 5)

In [82]:
xc=np.average(data,axis=0, weights=[1./4, 3./4,1]).reshape([1,2])

In [83]:
xc

array([[2.75, 3.75]])

In [84]:
yc=np.ones(2)

In [89]:
(xc*yc).shape

(1, 2)

In [28]:
weights_1=data.reshape([-1,1,2])
avg_weighting=[1./4, 3./4,1]
weights_1=np.average(local_1,axis=0,weights=avg_weighting_1)

NameError: name 'local_1' is not defined

In [34]:
local_1.shape

(66, 26)

In [50]:
Weighting_1.shape

(3,)

In [36]:
Weighting_1[1:,0].shape

(10,)

In [37]:
local_2.shape

(66, 26)

In [43]:
local_1.shape

(66, 26)

In [51]:
Weighting_1.shape


(3,)

In [48]:
print(data[1:,2:])

[]


In [65]:
data

array([[0, 1],
       [2, 3],
       [4, 5]])

In [66]:
data[1:,0]

array([2, 4])

In [67]:
data = range(1,9)

In [70]:
data = np.arange(9).reshape((3,3))

In [71]:
data

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [75]:
data[1:,0]

array([3, 6])

In [79]:
data[2:,0:]

array([[6, 7, 8]])

In [94]:
local_1d.shape

(12, 26)

In [82]:
avg_weighting_1d.shape

(2,)

In [83]:
print(avg_weighting_1d)

[0.02404309 0.00625549]


In [95]:
ll=local_1d.reshape([-1,2])

In [92]:
ll.shape

(156, 2)

In [96]:
llc=ll.reshape([12,26])

In [97]:
llc-local_1d

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 

In [99]:
lca=np.average(ll,axis=1,weights=avg_weighting_1d)

In [101]:
lcc=lca.reshape([6,26])

In [102]:
lcc

array([[ 2.55014886e-02,  5.27524665e-02, -1.22126338e-02,
        -4.20579900e-02,  7.47128086e-02, -9.08528634e-02,
        -3.56674858e-02,  1.54575845e-02,  4.04202678e-02,
         6.71619344e-02, -7.04969260e-02,  8.82855049e-02,
         3.19914166e-02,  1.74170172e-02, -7.87802927e-02,
        -1.99787394e-02,  8.24363647e-02,  1.14867249e-01,
        -1.30114549e-02,  2.41219415e-02, -7.10001055e-02,
         4.69713211e-02, -3.65647186e-02, -4.74943112e-02,
        -1.93085023e-02,  2.14384784e-01],
       [ 7.11451574e-02, -3.35031853e-02, -3.98009871e-02,
         4.26996119e-02, -2.30294394e-02, -3.05632668e-02,
         4.08858645e-02,  1.30932974e-02,  6.57687636e-02,
        -5.87753944e-02, -1.09535251e-01,  1.02703183e-01,
         2.08371592e-02,  6.78584279e-02,  2.47913219e-02,
         7.83008237e-02, -1.19347302e-01,  1.31706247e-02,
        -6.92202138e-02, -6.42787257e-02,  7.12870821e-02,
        -8.49309494e-02,  4.57110008e-02,  3.61527130e-02,
         7.27

In [104]:
local_1dl.shape

(312, 1)

SyntaxError: invalid syntax (<ipython-input-116-3b66d6d3ee38>, line 1)

In [117]:
a=[2 3]

SyntaxError: invalid syntax (<ipython-input-117-0066a6e896ad>, line 1)

In [119]:
a=np.array([[ 2.55014886e-02,  5.27524665e-02, -1.22126338e-02,
        -4.20579900e-02,  7.47128086e-02, -9.08528634e-02,
        -3.56674858e-02,  1.54575845e-02,  4.04202678e-02,
         6.71619344e-02, -7.04969260e-02,  8.82855049e-02,
         3.19914166e-02,  1.74170172e-02, -7.87802927e-02,
        -1.99787394e-02,  8.24363647e-02,  1.14867249e-01,
        -1.30114549e-02,  2.41219415e-02, -7.10001055e-02,
         4.69713211e-02, -3.65647186e-02, -4.74943112e-02,
        -1.93085023e-02,  2.14384784e-01],
       [ 7.11451574e-02, -3.35031853e-02, -3.98009871e-02,
         4.26996119e-02, -2.30294394e-02, -3.05632668e-02,
         4.08858645e-02,  1.30932974e-02,  6.57687636e-02,
        -5.87753944e-02, -1.09535251e-01,  1.02703183e-01,
         2.08371592e-02,  6.78584279e-02,  2.47913219e-02,
         7.83008237e-02, -1.19347302e-01,  1.31706247e-02,
        -6.92202138e-02, -6.42787257e-02,  7.12870821e-02,
        -8.49309494e-02,  4.57110008e-02,  3.61527130e-02,
         7.27094257e-02,  1.36126615e-02],
       [-1.67064908e-02,  4.60798322e-03, -1.08021788e-01,
         5.10525920e-02,  5.98024257e-02,  4.65812890e-02,
        -2.31957784e-02, -7.26181243e-02,  7.75987559e-02,
         2.49851261e-02, -5.51630016e-02,  3.69427327e-02,
        -3.47171013e-02, -1.75497931e-01,  6.13204050e-02,
        -1.06465674e-01,  9.03237416e-02, -1.56500426e-02,
        -2.19197783e-02,  1.19210617e-01, -8.03112912e-02,
        -1.80876952e-01,  1.71906381e-02,  5.74004455e-02,
         6.03544928e-02, -2.42024357e-04],
       [-1.15336708e-02,  7.22067280e-02, -1.27696250e-01,
        -4.86754186e-02,  1.92449040e-02, -1.11157898e-01,
        -9.42621534e-02, -8.35078062e-02,  1.13033741e-03,
         9.43133729e-02, -1.35477544e-01,  1.21744288e-01,
         3.48631772e-02,  1.30753623e-01, -1.29817135e-01,
        -8.16921332e-02,  1.15891475e-01,  1.21968307e-01,
         3.74755854e-02, -2.10518175e-02, -1.73153008e-02,
         2.06137914e-02, -4.83071111e-02, -9.32801281e-02,
        -5.98198829e-02,  2.99047720e-01],
       [ 4.43080354e-02, -2.31298609e-02, -8.78596236e-02,
         5.52608871e-02,  9.37111169e-02, -1.50608106e-02,
         6.43760490e-02,  2.68152141e-02,  1.10271987e-01,
        -8.31334471e-02, -1.90081889e-01,  7.93174935e-02,
         7.16765316e-03,  3.09634365e-02,  7.59668772e-03,
         1.59329355e-01, -2.23587184e-01,  8.32822804e-03,
        -1.69097883e-01, -1.23737978e-01,  1.15587915e-01,
        -1.60882146e-01,  5.46630027e-02,  6.24456092e-02,
         2.62631751e-02, -6.57363069e-02],
       [ 1.42215209e-01,  5.94310179e-02, -4.46812769e-02,
         6.99266457e-02,  1.88186442e-02,  1.27155600e-01,
         3.94511991e-02, -1.33996340e-01, -5.47956309e-04,
        -1.42325333e-02, -7.79504702e-02,  9.49384356e-04,
        -6.44518917e-02, -2.29280889e-01,  9.94059857e-02,
         1.44098823e-02,  7.57111014e-02, -6.40160422e-02,
        -5.20439192e-02,  2.64385034e-01, -1.02737277e-01,
        -3.24274562e-01, -7.10724008e-02,  7.95062441e-02,
         7.56820708e-02, -7.23131335e-02]])

In [120]:
print(lcc)

[[ 2.55014886e-02  5.27524665e-02 -1.22126338e-02 -4.20579900e-02
   7.47128086e-02 -9.08528634e-02 -3.56674858e-02  1.54575845e-02
   4.04202678e-02  6.71619344e-02 -7.04969260e-02  8.82855049e-02
   3.19914166e-02  1.74170172e-02 -7.87802927e-02 -1.99787394e-02
   8.24363647e-02  1.14867249e-01 -1.30114549e-02  2.41219415e-02
  -7.10001055e-02  4.69713211e-02 -3.65647186e-02 -4.74943112e-02
  -1.93085023e-02  2.14384784e-01]
 [ 7.11451574e-02 -3.35031853e-02 -3.98009871e-02  4.26996119e-02
  -2.30294394e-02 -3.05632668e-02  4.08858645e-02  1.30932974e-02
   6.57687636e-02 -5.87753944e-02 -1.09535251e-01  1.02703183e-01
   2.08371592e-02  6.78584279e-02  2.47913219e-02  7.83008237e-02
  -1.19347302e-01  1.31706247e-02 -6.92202138e-02 -6.42787257e-02
   7.12870821e-02 -8.49309494e-02  4.57110008e-02  3.61527130e-02
   7.27094257e-02  1.36126615e-02]
 [-1.67064908e-02  4.60798322e-03 -1.08021788e-01  5.10525920e-02
   5.98024257e-02  4.65812890e-02 -2.31957784e-02 -7.26181243e-02
   7.7

In [121]:
for a in range(2):
    lcc

In [136]:
print(local_1d[6:12])

[[ 0.01768857 -0.01386893 -0.05787283  0.07828453  0.20483754 -0.14427083
  -0.12151473 -0.07342291  0.10875512  0.15619998 -0.06772169  0.10989248
   0.0144843  -0.01735802 -0.00028552  0.02220344  0.04848413  0.03229894
   0.05158776  0.01750372 -0.02325308  0.00984654 -0.08888783 -0.06611041
  -0.00905672  0.02239462]
 [-0.09536852 -0.08657353  0.1550659   0.19547535  0.08783022  0.16807753
   0.10019957 -0.02169939  0.0125058  -0.17486179 -0.07878785  0.0398632
  -0.04675927 -0.12859745  0.07840628  0.08428011  0.08638876  0.02688441
  -0.12713748 -0.01174247 -0.01112341 -0.02266229  0.08517881 -0.00877523
  -0.01255788 -0.15785911]
 [ 0.08793698  0.17941409 -0.11079629 -0.08478138  0.08193868 -0.057032
  -0.17232981  0.17496922 -0.11473483  0.02887386 -0.05967157  0.12207091
   0.09262072  0.03032436  0.01064008  0.02517481  0.00513053  0.13756356
   0.11159786 -0.07491046  0.07062988 -0.07376934  0.05304116  0.04734782
   0.03982161 -0.01775067]
 [-0.0205675   0.12456083  0.03383

In [137]:
co_op_1d=co_op(1,env1,local_1d[6:12],True)

Reward: -983.216652366809 weights: [[ 0.12053824 -0.0542102   0.04375696  0.07326041  0.18777045 -0.01084289
  -0.18048491 -0.12218386 -0.01913746  0.02438141 -0.03708293  0.0495879
   0.08974725 -0.11254845 -0.06280241  0.08798216  0.19245732  0.09228992
   0.14546658  0.14181475  0.07132268 -0.06393885 -0.06465451 -0.03388466
  -0.07412814  0.04251796]
 [-0.23928396 -0.16465801  0.16757368  0.04708333  0.00642335  0.16599752
  -0.00333622 -0.02694    -0.04733629 -0.20601033 -0.06674966  0.0606826
  -0.05250235 -0.12665089  0.0314125   0.12623149  0.18705242  0.03237166
  -0.11680029 -0.0937602   0.02884715 -0.08134832 -0.00290495  0.02629138
  -0.03239689 -0.09389832]
 [ 0.19550746  0.15704235 -0.10842469  0.00294803  0.17303028  0.00884041
  -0.08839941  0.12583264 -0.04700209  0.13608547  0.09812138  0.07116155
   0.12573096  0.062986   -0.01868793  0.00631055  0.05789459  0.11171928
  -0.05103008 -0.05553903  0.03706661 -0.11821281 -0.06453803  0.10012862
  -0.11870932 -0.13460926

In [138]:
print(co_op_1d.reward_evaluation)

-983.216652366809


In [139]:
co_op_2d=co_op(2,env2,local_1d[6:12],True)

Reward: -930.7091690964162 weights: [[ 0.17991415 -0.04768459  0.01547834  0.13406077  0.20962484 -0.01960424
  -0.22389735 -0.09206547  0.13435573  0.01358195 -0.17474389 -0.16797908
   0.14008142 -0.29638686 -0.04048417  0.16468398  0.22164819  0.24494682
   0.10255255  0.15129299 -0.01941616 -0.14912123 -0.13255396  0.08458331
   0.05914014 -0.00946631]
 [-0.12964009 -0.14980822  0.23762866  0.00060484  0.04955396  0.18330745
  -0.08865225 -0.09337893 -0.12096981 -0.21848841  0.05422561  0.16100039
   0.0545098  -0.07461298  0.07388923  0.01999074  0.20354373  0.12840941
  -0.03639626 -0.15372336  0.0425371  -0.06439084  0.02751194  0.01018968
   0.00806973 -0.1210308 ]
 [ 0.13015916  0.16393904 -0.12509521  0.12146984  0.25294469 -0.21987653
   0.03735035  0.13650199  0.03547914  0.2685952   0.27619694  0.14037812
   0.11046867  0.02369298 -0.03263511  0.0227125   0.04891924  0.29848538
   0.08784287 -0.02120935  0.17704153 -0.00186039 -0.07817111  0.17930527
   0.10568872 -0.10947

In [140]:
print(co_op_2d.reward_evaluation)

-930.7091690964162


In [12]:
import pandas as pd 
df1 = pd.DataFrame(local_1)
df1.to_csv("local_1.csv")
df1 = pd.DataFrame(local_2)
df1.to_csv("local_2.csv")
df1 = pd.DataFrame(local_1d)
df1.to_csv("local_1d.csv")

In [13]:
df1 = pd.DataFrame(local_2d)
df1.to_csv("local_2d.csv")

In [14]:
df1 = pd.DataFrame(weights_1)
df1.to_csv("weights_1.csv")
df1 = pd.DataFrame(weights_2)
df1.to_csv("weights_2.csv")

In [33]:
class co_op_video():
    def __init__(self,hp_seed,env,weights=None,update=False):
        #self.test=hp_seed
        self.hp=Hp()
        self.hp.seed=hp_seed
        #self.hp.seed=int(time.time())
        np.random.seed(self.hp.seed)
        #self.env=gym.make(self.hp.env_name)
        #self.work_dir=mkdir('exp','brs_video{}'.format(hp_seed))
        #self.monitor_dir=mkdir(self.work_dir,'monitor')
        #self.env=wrappers.Monitor(self.env,self.monitor_dir,force=True)
        self.nb_inputs=env.observation_space.shape[0]
        self.nb_outputs=env.action_space.shape[0]
        self.policy=Policy(self.nb_inputs,self.nb_outputs)
        if update:
            self.policy.theta=weights
        self.normalizer=Normalizer(self.nb_inputs)
    #return env,policy,normalizer
    #print(hp.nb_directions)
        self.reward_evaluation,self.weights,self.sigma_r=train(env,self.policy,self.normalizer,self.hp)
    #return reward_evaluation,weights

In [53]:
hp=Hp()
hp.seed=1
np.random.seed(hp.seed)
env1=gym.make(hp.env_name)
work_dir=mkdir('exp','brs_video1')
monitor_dir=mkdir(work_dir,'monitor')
env1=wrappers.Monitor(env1,monitor_dir,force=True)
co_op_video_1=co_op_video(1,env1,local_1[4*6:5*6],True)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WalkerBase::__init__ start
Reward: -827.3789871987317 weights: [[ 0.24368639 -0.01850468  0.00979615  0.23369479  0.0249999  -0.19418553
  -0.23897502  0.10002709 -0.1439752  -0.18555511 -0.00359751  0.1582014
  -0.12697571  0.11300304  0.18920173 -0.0555842   0.39291617  0.09129656
   0.00535168 -0.04970305  0.22443138  0.12328946 -0.08653993 -0.0763396
   0.17706982 -0.08076734]
 [-0.03761038  0.08145479 -0.00936432 -0.01933868  0.17562054 -0.20388318
   0.01759076 -0.09244444  0.27004391  0.27095345  0.18341145 -0.02973915
   0.20427268  0.06769463 -0.27037765 -0.05125538 -0.08235912  0.0999997
  -0.1048566  -0.04692073  0.00102942 -0.07872045  0.05397767 -0.04177909
   0.08379773  0.20930829]
 [ 0.29026542 -0.06359639  0.02171806  0.2630452   0.11287797  0.12001089
   0.12740851  0.10126

In [16]:
weights_1.shape

(6, 26)

In [52]:
hp=Hp()
hp.seed=2
np.random.seed(hp.seed)
env2=gym.make(hp.env_name)
work_dir=mkdir('exp','brs_video2')
monitor_dir=mkdir(work_dir,'monitor')
#env2=wrappers.Monitor(env2,monitor_dir,video_callable=lambda episode_id: episode_id%20==0,force=True)
env2=wrappers.Monitor(env2,monitor_dir,force=True)
co_op_video_2=co_op_video(2,env2,local_2[2*6:3*6],True)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WalkerBase::__init__ start
Reward: 443.9659915450178 weights: [[-2.82932345e-02  1.69716662e-02  6.88499297e-02  1.17723318e-01
  -2.46454653e-01  3.50798269e-02 -2.51633555e-02 -5.03678291e-02
  -4.47236646e-01  5.12845706e-01 -2.43701358e-02  2.31253237e-01
  -3.29466147e-01  6.97975476e-01 -1.45487374e-01  1.10519404e-01
   8.83872429e-02  3.28536541e-01 -1.49753820e-01  1.41796578e-01
  -7.60052164e-02  7.82811259e-02 -3.16098283e-01 -1.78193430e-01
   1.61571039e-01 -1.10669264e-01]
 [-1.55649785e-01 -2.50038766e-02  1.64955324e-01 -1.87675469e-01
   1.08083270e-01 -5.87141735e-02  3.18232150e-01 -9.24704697e-02
   1.49369323e-01  4.18876386e-01  8.29534689e-02  4.75429811e-02
  -2.13670348e-01  2.83677230e-01 -2.25839271e-01 -1.16826345e-02
  -4.36047044e-02  5.66433171e-03 -2.07936319

In [34]:
local_2[1*6:2*6]

array([[ 2.32232678e-01, -4.46294611e-02,  7.33115307e-02,
        -2.55555435e-04, -6.49862924e-02, -2.00768774e-03,
         5.86184496e-02, -1.24463211e-01, -1.01761244e-01,
         9.28429803e-02,  1.19910844e-01,  7.09492252e-02,
        -1.77603974e-01,  1.62759596e-01, -1.50328326e-02,
        -2.59148196e-02, -9.63106968e-02,  2.72057354e-01,
        -1.29673081e-01,  8.79883911e-03, -1.32976706e-01,
         1.91774794e-01, -3.89397103e-01, -2.07033826e-01,
         1.38243647e-01, -1.73763771e-01],
       [-6.52636032e-02, -3.53026761e-02,  9.06823498e-02,
        -1.18949190e-01,  1.02420081e-01,  2.80554895e-02,
         9.30463395e-02, -5.00533145e-02, -1.91179104e-02,
         4.09154236e-01, -1.41339506e-01,  8.83827047e-02,
        -2.06168811e-01,  4.67498098e-01, -3.58700201e-01,
         1.08703315e-04, -8.98037023e-02,  4.94515436e-02,
         9.39033845e-02, -2.69435693e-02, -3.00612602e-01,
         5.99916121e-02,  1.04098182e-03,  2.58797522e-01,
         9.22

In [35]:
local_1.shape

(60, 26)

In [37]:
local_1[0:6]

array([[ 0.09857533, -0.04321631, -0.03579079,  0.06776905,  0.01338706,
        -0.13903269, -0.00242192,  0.05972336, -0.15422582,  0.01305957,
        -0.07190221, -0.01871827, -0.1421831 ,  0.16715691, -0.07673198,
         0.05802091,  0.07947249,  0.05865806, -0.05545493,  0.05348605,
         0.14300429,  0.06077916, -0.03779264, -0.01950044, -0.03466338,
        -0.00950199],
       [-0.0497492 ,  0.19209283, -0.07219576,  0.16867341, -0.04184084,
        -0.06447137,  0.07210458, -0.01714962,  0.0948479 ,  0.15197702,
         0.0789123 , -0.0202926 ,  0.02299222,  0.02421136,  0.08787295,
        -0.048247  , -0.07630673, -0.01609258, -0.00640992, -0.07216009,
         0.08545051,  0.00705288,  0.0250386 , -0.09183332,  0.02590828,
         0.21476614],
       [ 0.01999133, -0.05347642, -0.02389055,  0.22999822,  0.01882118,
        -0.02623535,  0.02059167,  0.03866346,  0.14577752,  0.0438011 ,
         0.22278041, -0.05491971,  0.01836825,  0.18949354,  0.06089882,
       

In [42]:
local_1[6:12]

array([[ 0.12907875, -0.125525  ,  0.01044309,  0.19039627, -0.04329899,
        -0.04476498, -0.05654388,  0.02142521, -0.22490023,  0.0619913 ,
        -0.09663566, -0.0830068 , -0.21288082,  0.06616003, -0.16549157,
         0.00748878,  0.08611463,  0.10818784,  0.05138643,  0.08946653,
         0.08456192, -0.00770237, -0.04667985, -0.02740411,  0.01382517,
        -0.0522625 ],
       [-0.06028353,  0.1765072 , -0.06543631,  0.01664835, -0.0662191 ,
        -0.08737367,  0.11751286, -0.00829731,  0.0934754 ,  0.07858246,
        -0.06221906,  0.01632981, -0.0013863 ,  0.03802935, -0.03041743,
        -0.07060298, -0.00204251,  0.03198242, -0.06348108, -0.08212587,
         0.02962125, -0.08194855, -0.00144078, -0.08748373,  0.23828416,
         0.38826749],
       [ 0.13427286, -0.05067705, -0.10046695,  0.42125365,  0.03619262,
         0.01841114,  0.20046283,  0.00246562,  0.12389836,  0.08870726,
         0.20946089, -0.07534692,  0.00430329,  0.20489728,  0.10712997,
       

In [49]:
print(env2.observation_space)

Box(26,)
